In [126]:
import os
import json
import boto3
import wandb
import pandas as 
from collections import defaultdict

In [127]:
# Creating a run and using the downloaded data
run = wandb.init(project="data-engineering", group="EDA", save_code=True)
data = wandb.use_artifact("raw_data:latest").download()

wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: Downloading large artifact raw_data:latest, 170.82MB. 79 files... Done. 0:0:0


In [143]:
# Creating output dir for the processed data
output_dir= "processed_data"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# Looping over all the files in the directory
for root,dirs,files in os.walk(data):
    for file in files:
        # Opening each file seperately
        with open(os.path.join(data,file), mode='r') as f:
            extracted_data = json.load(f)
            entries = extracted_data["entry"]
            
            # Creating set & dictionary for each file
            resource_types = set()
            entry_resources = defaultdict(list)
        
            # Extracting data to dictionary based on resource type for each file
            for resource in entries:
                resource_type = resource["resource"]["resourceType"]
                resource_types.add(resource_type)
                resource_file_path = output_dir +"/"+ f"{resource_type}.json"
                entry_resources[resource_file_path].append(resource)
            
        
            for resource_file_path, resources in entry_resources.items():
                if os.path.exists(resource_file_path):
                    #Getting existing records of each resource of each file
                    with open(resource_file_path,mode='r') as i:
                        contents = json.load(i)
                
                    contents.extend(resources)
                else:
                    contents = resources
                
                # save it to disk
                with open(resource_file_path,mode="w") as j:
                    json.dump(contents, j)  

In [144]:
# Finishing the run
run.finish()